In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
import matplotlib
import matplotlib.pylab as pylab
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.patches as mpatches
import numpy as np
import numpy.linalg as la

def add_relative_to_current_source_file_path_to_sys_path(relpath):
    import os, sys, inspect
    path = os.path.realpath(os.path.abspath(os.path.join(os.path.split(inspect.getfile(inspect.currentframe()))[0],relpath)))
    if path not in sys.path:
        sys.path.insert(0,path)

add_relative_to_current_source_file_path_to_sys_path("../../shims/toric")   

import toric
import toric.samples
import toric.shims

# An alternative to averaging trajectories: The magic $g_i(\vec{\sigma}_A,\vec{\sigma}_B,u)$ function

Recall from the notebook *"NJ - 06 - Toric Interpolation Proof of Min Dist Constraint Maintenance"*

$$
\begin{align}
\vec{\sigma}_A(u) & = \vec{A} + d_A(u)\vec{v}_A(u) \\
\vec{\sigma}_B(u) & = \vec{B} + d_B(u)\vec{v}_B(u) \\
\end{align}
$$

Notice that $\vec{\sigma}_A(u)$ respects the minimum distance bounds on A, and $\vec{\sigma}_B(u)$ respects the minimum distance bounds on B. Thus, we are only concerned with $\vec{\sigma}_A(u)$ coming close to B and vice versa.

Let's suggest a different way to combine these two trajectories. Intuitively, we want a function that, if both $\vec{\sigma}_A(u)$ and $\vec{\sigma}_B(u)$ are far away from A and B, just averages the two. If $\vec{\sigma}_A(u)$ gets close to B, it picks only $\vec{\sigma}_B(u)$, and vice versa.

Lastly, this function needs to guarantee C4-or-better continuity to generate quadrotor feasible paths.

Let's formulate this function explicitly. We call it $g_i(\vec{\sigma}_A,\vec{\sigma}_B,u)$

$$
\vec{\sigma}(u) = g_A(\vec{\sigma}_A,\vec{\sigma}_B,u)\cdot\vec{\sigma}_A(u) + g_B(\vec{\sigma}_B,\vec{\sigma}_A,u)\cdot\vec{\sigma}_A(u)
$$

** Restriction: We still consider the case where $\vec{C}_0$ and $\vec{C}_1$ are both on the same side of the line $AB$. 

*We might be able to relax this restriction, but lets keep it in place for the moment*